In [1]:
!pip install catboost mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 1.0 MB/s  0:01:32m0:00:0100:030m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 1.1 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.7/774.7 kB 594.3 kB/s  0:00:01eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 1.1 MB/s  0:00:45m0:00:0100:020m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 752.3 kB/s  0:00:04 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 1.0 MB/s  0:00:0036m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 1.2 MB/s  0:00:08m0:00:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42/42 [mlflow] [mlflow] [mlflow-skinny]]c-conventions]


In [3]:
import os
import json
import numpy as np
import pandas as pd

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

from catboost import CatBoostClassifier
import mlflow
import mlflow.catboost
import joblib


# ---------------- CONFIG ----------------
TRAIN_CSV = "../data/processed/train.csv"
VAL_CSV = "../data/processed/val.csv"
TEST_CSV = "../data/processed/test.csv"

EXPERIMENT_NAME = "customer-churn-merged-split"
OUT_DIR = "../artifacts"

RANDOM_STATE = 42

TARGET = "Churn"
ID_COL = "CustomerID"

CAT_COLS = ["Gender", "Subscription Type", "Contract Length"]
NUM_COLS = [
    "Age","Tenure","Usage Frequency",
    "Support Calls","Payment Delay",
    "Total Spend","Last Interaction"
]

DROP_FEATURES = []   # optionally add ["Total Spend"] etc.


# ---------------- HELPERS ----------------
def split_xy(df):
    """Separate features and target"""
    y = df[TARGET].values
    X = df.drop(columns=[TARGET, ID_COL], errors="ignore")
    if DROP_FEATURES:
        X = X.drop(columns=[c for c in DROP_FEATURES if c in X.columns])
    return X, y

def evaluate(y_true, proba, threshold=0.5):
    """Standard binary classification metrics"""
    pred = (proba >= threshold).astype(int)
    return {
        "accuracy": accuracy_score(y_true, pred),
        "precision": precision_score(y_true, pred, zero_division=0),
        "recall": recall_score(y_true, pred, zero_division=0),
        "f1": f1_score(y_true, pred, zero_division=0),
        "roc_auc": roc_auc_score(y_true, proba),
        "pred_pos_rate": pred.mean(),
        "confusion_matrix": confusion_matrix(y_true, pred).tolist()
    }


# ---------------- MAIN ----------------
def main():
    os.makedirs(OUT_DIR, exist_ok=True)

    # ---- Load processed datasets ----
    df_train = pd.read_csv(TRAIN_CSV)
    df_val = pd.read_csv(VAL_CSV)
    df_test = pd.read_csv(TEST_CSV)

    # ---- Split features and target ----
    X_train, y_train = split_xy(df_train)
    X_val, y_val = split_xy(df_val)
    X_test, y_test = split_xy(df_test)

    cat_cols = [c for c in CAT_COLS if c in X_train.columns]

    # ---- Model ----
    model = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        iterations=2000,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=5,
        random_seed=RANDOM_STATE,
        early_stopping_rounds=100,
        verbose=200
    )

    # ---- MLflow ----
    mlflow.set_experiment(EXPERIMENT_NAME)
    with mlflow.start_run():
        mlflow.log_params({
            "model": "CatBoostClassifier",
            "iterations": 2000,
            "learning_rate": 0.05,
            "depth": 6,
            "drop_features": ",".join(DROP_FEATURES) if DROP_FEATURES else "NONE"
        })

        # Train with validation
        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            cat_features=cat_cols,
            use_best_model=True
        )

        # Test evaluation
        test_proba = model.predict_proba(X_test)[:, 1]
        metrics = evaluate(y_test, test_proba)

        print("\nTEST METRICS @0.5")
        for k, v in metrics.items():
            if k != "confusion_matrix":
                print(f"{k}: {v}")
        print("Confusion matrix:")
        print(np.array(metrics["confusion_matrix"]))

        # Log metrics
        for k, v in metrics.items():
            if k != "confusion_matrix":
                mlflow.log_metric(k, v)

        # Save artifacts
        report_path = os.path.join(OUT_DIR, "test_report.json")
        with open(report_path, "w") as f:
            json.dump(metrics, f, indent=2)
        mlflow.log_artifact(report_path)

        model_path = os.path.join(OUT_DIR, "catboost_model.cbm")
        model.save_model(model_path)
        mlflow.log_artifact(model_path)

        meta_path = os.path.join(OUT_DIR, "model_meta.joblib")
        joblib.dump({
            "cat_cols": cat_cols,
            "drop_features": DROP_FEATURES,
            "target": TARGET
        }, meta_path)
        mlflow.log_artifact(meta_path)

        mlflow.catboost.log_model(model, name="model")

    print("\nDONE. Using processed train/val/test splits + MLflow tracking.")

if __name__ == "__main__":
    main()


0:	test: 0.9316922	best: 0.9316922 (0)	total: 83.9ms	remaining: 2m 47s
200:	test: 0.9540092	best: 0.9540092 (200)	total: 10.7s	remaining: 1m 35s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9541535515
bestIteration = 272

Shrink model to first 273 iterations.

TEST METRICS @0.5
accuracy: 0.931721462362186
precision: 0.8988456175621777
recall: 0.9882350844043566
f1: 0.9414231980777231
roc_auc: 0.9534640983102148
pred_pos_rate: 0.61041942954415
Confusion matrix:
[[38704  6239]
 [  660 55439]]

DONE. Using processed train/val/test splits + MLflow tracking.
